# Memory: A Deep Dive: Part III (Schema Based Autonomous Memory Management)


## Overview
This code implements a workflow-based memory management system that combines LangGraph and GPT-4 mini to update existing memories based on new interactions. The system takes previously generated memories and uses a graph-based workflow to identify relevant memories that need updating, then modifies them while maintaining their hierarchical organization (type → label → content → context). This approach ensures that memories evolve coherently as new information becomes available.

## Motivation
Several key factors motivate this implementation:

1. **Dynamic Memory Maintenance**
   - Memories need to evolve with new information
   - Manual memory updates are error-prone and inconsistent
   - LLMs can intelligently merge new information with existing context

2. **Intelligent Update Flow**
   - Need for systems that identify relevant memories to update
   - Importance of maintaining memory consistency
   - Value of tracking memory modification history

3. **Context Preservation**
   - Need for updates that preserve important historical information
   - Importance of maintaining memory structure during updates
   - Benefits of contextual memory evolution

4. **Automated Conflict Resolution**
   - Recognition that new information may contradict existing memories
   - Value of intelligent conflict resolution
   - Importance of maintaining memory coherence

## Key Components
1. **State Management System**: 
   - Maintains workflow state using Pydantic models
   - Tracks interaction, existing memories, and update indexes
   - Ensures data integrity during updates

2. **LLM Integration**: 
   - Leverages OpenAI's GPT-4 mini model
   - Updates memories while preserving structure:
     - Identifies relevant memories to update
     - Modifies content while maintaining context
     - Preserves memory schema compliance

3. **Graph-based Workflow**: 
   - Uses LangGraph's StateGraph for orchestration
   - Implements memory search and modification nodes
   - Controls update flow through conditional edges

4. **Memory Structure**:
   - Preserves hierarchical organization during updates
   - Maintains contextual information
   - Ensures schema compliance

## Method
The system follows this workflow:

1. **Initialization**:
   - Takes new interaction and existing memories as input
   - Sets up state management system
   - Initializes LLM client

2. **Memory Search**:
   - Analyzes interaction against existing memories
   - Identifies memories requiring updates
   - Tracks memory indexes for modification

3. **Memory Modification**:
   - Updates identified memories
   - Preserves relevant historical context
   - Maintains memory structure

4. **Flow Control**:
   - Manages update process
   - Routes between search and modification
   - Determines workflow completion

## Visual Overview
A flowchart representing the design and flow of the workflow.

<div style="max-width:400px;">
    
![image.png](../images/memory_management.png)
    
</div>

## Conclusion
This implementation demonstrates a practical approach to automated memory updating. The system combines graph-based workflow management with LLM capabilities, allowing for intelligent memory evolution while maintaining structural integrity.

Key advantages include:
- Intelligent memory identification for updates
- Context-aware modification process
- Structured update workflow
- Conditional processing flow

Future improvements could focus on:
- Memory version control
- Update conflict resolution
- Update history tracking
- Memory consistency validation

This system provides a foundation for building more sophisticated memory management systems, particularly in applications requiring dynamic knowledge maintenance and evolution.

# Dependencies and Imports
Install dependencies and import libraries.

In [1]:
%%capture

!pip install langgraph
!pip install langgraph-sdk
!pip install langgraph-checkpoint-sqlite
!pip install langchain-community
!pip install langchain-core
!pip install langchain-openai

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

from pydantic import BaseModel
from typing import Optional

import json
import os


## Clients
Import API keys and instantiate clients.

In [10]:
os.environ['OPENAI_API_KEY'] = 'YOUR-API-KEY'
llm = ChatOpenAI(model='gpt-4o-mini')

## Define Agent State
We'll define the state that our agent will maintain throughout its operation.


In [4]:
class State(BaseModel):
    interaction: str
    memory_schema: list[str] = []
    memory_labels: list[str] = []
    memories: list = []
    memory_indexes_to_update: list = []


## Define Node Functions
Now we'll define the main node functions that our agent will use: generate_schema and update_instructions.


In [5]:
def memory_search_node(state: State):
    ''' Find memories relevant to the current interaction '''
    prompt = ChatPromptTemplate.from_template(
        'Given the following interaction and existing memories, identify which memories are relevant and need updating:'
        'Interaction: {interaction}'
        'Existing Memories: {memories}'
        'The response should be a list of integers representing memory indexes to be updated'
        'Response Format: [memory_index, ... , memory_index]'
    )
    
    message = HumanMessage(content=prompt.format(
        interaction=state.interaction,
        memories=state.memories
    ))
    
    response = llm.invoke([message]).content.strip()
    memory_indexes_to_update = json.loads(response)
    state.memory_indexes_to_update = memory_indexes_to_update

    return state


def memory_modification_node(state: State):
    ''' Modify relevant memories based on new interaction '''
    prompt = ChatPromptTemplate.from_template(
        'Update the following memories based on the new interaction:'
        'Interaction: {interaction}'
        'Memory to Update: {memory_to_update}'
        'Response Format: # memory_label ## updated_memory_content ### updated_memory_context'
    )
    memory_to_update_index = state.memory_indexes_to_update.pop(0)
    memory_to_update = state.memories[memory_to_update_index]
    message = HumanMessage(content=prompt.format(
        interaction=state.interaction,
        memory_to_update=memory_to_update,
    ))
    
    response = llm.invoke([message]).content.strip()
    
    state.memories[memory_to_update_index] = response
        
    return state
    

## Define Edge Functions
Now we'll define the conditional edge function that our agent will use to control the workflow.

In [6]:
def modification_router(state: State):
    if state.memory_indexes_to_update:
        return 'memory_modification_node'
    else:
        return END

## Build Workflow
Now we'll create our workflow and compile it.


In [7]:
builder = StateGraph(State)

# Add nodes to the graph
builder.add_node('memory_search_node', memory_search_node)
builder.add_node('memory_modification_node', memory_modification_node)


# Add edges to the graph
builder.set_entry_point('memory_search_node')
builder.add_conditional_edges('memory_search_node', modification_router)
builder.add_conditional_edges('memory_modification_node', modification_router)

# Compile the graph
graph = builder.compile()

# Main Function
Define the function that runs the instanciates the workflow and its state.

In [8]:
# Updated main function
def run_memory_system(interaction: str, memory_schema: list, memories: list = []):
    
    state = State(
        interaction=interaction, 
        memory_schema=memory_schema,
        memories=memories
    )
    
    for output in graph.stream(state):
        last_state = output
        pass

    return last_state
    


# Run Program
Instanciate the main function and observe outputs.

In [9]:
memory_schema = [
    '# episodic ## tutoring_sessions ### A record of individual tutoring sessions, including details such as the date, subject matter, student feedback, and specific challenges faced during the session. This memory helps the agents recall past interactions and improve future sessions based on previous experiences.',
    '# semantic ## subject_knowledge ### A structured repository of knowledge on various subjects, including definitions, key concepts, and common problems encountered in tutoring. This memory allows agents to provide accurate information and explanations to students.',
    '# procedural ## tutoring_process ### A step-by-step guide outlining the approach agents should take when conducting a tutoring session, including preparation, delivery of content, engagement strategies, and assessment methods. This memory ensures a consistent and effective tutoring experience across all agents.',
    '# prospective ## future_goals ### A list of objectives and milestones for both students and agents, including skill acquisition targets, progress tracking, and plans for future sessions. This memory helps agents stay focused on the educational journey of their students and encourages proactive planning.',
]

memories =  ["# semantic  \n## My favorite color is blue.  \n### This memory captures the user's personal preference, indicating that blue is their favorite color. This information can enhance interactions by allowing for personalization in future conversations, such as tailoring recommendations or discussions around topics related to the color blue."]
# Update interaction
updated_memories = run_memory_system("Actually, my favorite color is red", memory_schema, memories)
print(updated_memories)

{'memory_modification_node': {'interaction': 'Actually, my favorite color is red', 'memory_schema': ['# episodic ## tutoring_sessions ### A record of individual tutoring sessions, including details such as the date, subject matter, student feedback, and specific challenges faced during the session. This memory helps the agents recall past interactions and improve future sessions based on previous experiences.', '# semantic ## subject_knowledge ### A structured repository of knowledge on various subjects, including definitions, key concepts, and common problems encountered in tutoring. This memory allows agents to provide accurate information and explanations to students.', '# procedural ## tutoring_process ### A step-by-step guide outlining the approach agents should take when conducting a tutoring session, including preparation, delivery of content, engagement strategies, and assessment methods. This memory ensures a consistent and effective tutoring experience across all agents.', '#